# 2D人体姿态追踪 预训练模型预测

参考文档：https://github.com/open-mmlab/mmpose/blob/master/demo/docs/2d_pose_tracking_demo.md

作者：同济子豪兄 2022-06-06

如果报错`CUDA out of memory.`则重启前面几个代码的`kernel`即可。

## 进入 MMPose 主目录

In [1]:
import os
os.chdir('mmpose')

## 2D Top-Down Video Human Pose Tracking

mmpose原生追踪算法原理：https://github.com/open-mmlab/mmpose/blob/master/mmpose/apis/inference_tracking.py#L39-L116

（根据iou或者oks算一个贪心的匹配，极其简易的追踪算法）

### 单帧视频预测

--bbox-thr 目标检测框阈值，默认为0.3

--kpt-thr 关键点检测阈值，默认为0.3

In [2]:
!python demo/top_down_pose_tracking_demo_with_mmdet.py \
        demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
        https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
        configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/res50_coco_256x192.py \
        https://download.openmmlab.com/mmpose/top_down/resnet/res50_coco_256x192-ec54d7f3_20200709.pth \
        --video-path data/mot_people_short.mp4 \
        --out-video-root outputs/D1/D1_1_top_down_tracking_single_frame


Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/resnet/res50_coco_256x192-ec54d7f3_20200709.pth
Downloading: "https://download.openmmlab.com/mmpose/top_down/resnet/res50_coco_256x192-ec54d7f3_20200709.pth" to /home/featurize/.cache/torch/hub/checkpoints/res50_coco_256x192-ec54d7f3_20200709.pth
100%|█████████████████████████████████████████| 130M/130M [00:01<00:00, 112MB/s]
Running inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 4.3 task/s, elapsed: 23s, ETA:     0s


### 多帧视频预测（速度会慢一些）

使用`--use-multi-frames`参数，将视频前后多帧画面输入模型用于姿态预测。

使用`--online`参数，仅输入该帧之前的帧，不输入该帧之后的帧。

In [6]:
!python demo/top_down_pose_tracking_demo_with_mmdet.py \
        demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
        https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
        configs/body/2d_kpt_sview_rgb_vid/posewarper/posetrack18/hrnet_w48_posetrack18_384x288_posewarper_stage2.py \
        https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth  \
        --video-path data/mot_people_short.mp4 \
        --out-video-root outputs/D1/D1_2_top_down_tracking_multi_frames \
        --use-multi-frames \
        --online

Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
Running inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 0.5 task/s, elapsed: 181s, ETA:     0s


## 2D Bottom-Up Video Human Pose Tracking

In [3]:
!python demo/bottom_up_pose_tracking_demo.py \
        configs/body/2d_kpt_sview_rgb_img/associative_embedding/coco/hrnet_w32_coco_512x512.py \
        https://download.openmmlab.com/mmpose/bottom_up/hrnet_w32_coco_512x512-bcb8c247_20200816.pth \
        --video-path data/mot_people_short.mp4 \
        --out-video-root outputs/D1/D1_5_bottom_up_tracking_video

load checkpoint from http path: https://download.openmmlab.com/mmpose/bottom_up/hrnet_w32_coco_512x512-bcb8c247_20200816.pth
Downloading: "https://download.openmmlab.com/mmpose/bottom_up/hrnet_w32_coco_512x512-bcb8c247_20200816.pth" to /home/featurize/.cache/torch/hub/checkpoints/hrnet_w32_coco_512x512-bcb8c247_20200816.pth
100%|█████████████████████████████████████████| 109M/109M [00:01<00:00, 114MB/s]
[                                                  ] 0/99, elapsed: 0s, ETA:/home/featurize/work/MMPose教程/mmpose/mmpose/core/post_processing/group.py:240: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  y = ind // W
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 99/99, 0.2 ta

## 基于 MMTracking 的 2D Top-Down Video Human Pose Tracking

### 安装MMTracking

MMTracking教程：https://www.bilibili.com/video/BV1s44y1g75J

### 单帧视频预测

In [4]:
!python demo/top_down_pose_tracking_demo_with_mmtracking.py \
        demo/mmtracking_cfg/tracktor_faster-rcnn_r50_fpn_4e_mot17-private.py \
        configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/res50_coco_256x192.py \
        https://download.openmmlab.com/mmpose/top_down/resnet/res50_coco_256x192-ec54d7f3_20200709.pth \
        --video-path data/mot_people_short.mp4 \
        --out-video-root outputs/D1/D1_3_mmtracking_single_frame

Initializing model...
/home/featurize/work/MMPose教程/mmtracking/mmtrack/models/mot/tracktor.py:28: UserWarning: DeprecationWarning: pretrains is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrains is deprecated, '
2022-07-06 16:11:03,055 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth'}
2022-07-06 16:11:03,055 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth
2022-07-06 16:11:03,056 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth
Downloading: "https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth" to /home/featurize/.cache/torch/hub/checkpoints/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.p

### 多帧视频预测

In [5]:
!python demo/top_down_pose_tracking_demo_with_mmtracking.py \
    demo/mmtracking_cfg/tracktor_faster-rcnn_r50_fpn_4e_mot17-private.py \
    configs/body/2d_kpt_sview_rgb_vid/posewarper/posetrack18/hrnet_w48_posetrack18_384x288_posewarper_stage2.py \
    https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth  \
    --video-path data/mot_people_short.mp4 \
    --out-video-root outputs/D1/D1_4_mmtracking_multi_frames \
    --use-multi-frames \
    --online

Initializing model...
/home/featurize/work/MMPose教程/mmtracking/mmtrack/models/mot/tracktor.py:28: UserWarning: DeprecationWarning: pretrains is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrains is deprecated, '
2022-07-06 16:12:15,434 - mmtrack - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth'}
2022-07-06 16:12:15,434 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth
2022-07-06 16:12:15,434 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-ffa52ae7.pth
2022-07-06 16:12:15,605 - mmtrack - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/reid/reid_r50_6e_mot17-4bf6b63d.pth'}
202